<a href="https://colab.research.google.com/github/catafest/colab_google/blob/master/catafest_042.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LLaMA(Large Language Model Meta AI) is a collection of state-of-the-art foundation language models ranging from 7B to 65B parameters.

You can read more ono [this paper](https://arxiv.org/pdf/2302.13971.pdf).

Let test with an GPU and this source code for a simple task: "*Tell me a story.*"

You can change the this line of source code with another one :

`instruction = "Tell me a story."`

In [7]:
%%capture
%pip install transformers SentencePiece accelerate

In [8]:
import transformers, torch
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LlamaForCausalLM.from_pretrained(
        "decapoda-research/llama-7b-hf",
        load_in_8bit=False,
        torch_dtype=torch.float16,
        device_map="auto",
    )

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [17]:
instruction = "Tell me a story."

inputs = tokenizer(
    f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction: {instruction}
### Response:""",
    return_tensors="pt",
)
input_ids = inputs["input_ids"].to("cuda")

generation_config = transformers.GenerationConfig(
    do_sample=True,
    temperature=0.1,
    top_p=0.75,
    top_k=80,
    repetition_penalty=1.5,
    max_new_tokens=128,
)

with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config,
    )
output_text = tokenizer.decode(
    generation_output[0].cuda(), skip_special_tokens=True
).strip()
print(output_text)

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction: Tell me a story.
### Response: I was walking down Main Street when suddenly, out of nowhere came this huge dog! It ran right at my face and knocked over all these people who were standing in front of it’s store window display!! The owner yelled “Get back here!” but he didn't listen to him so we had no choice except for us kids running away from there as fast our legs could carry them!!! We got home safely though because mom told dad about what happened on his way off work today…..
